Regressions between ERA5 (dif. variables) and Cloud Index (CI) in Corrientes Argentina

Las anomalias de CI se generaron asi:

* Calculo el CI mensuales para los meses con menos del 20% de datos faltantes
* Calculo las anomalias de CI mensuales (le resto a las medias mensuales la CI climatologica de cada mes calculada sobre el periodo comun con el satelite: fecha_inicio_climatologia_str = "1983-12-15"
fecha_final_climatologia_str = "2016-11-15")
* Todo se guarda en csvs: cada columna es una estacion. En: "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/paper"
TCC_monthly_CI_SMN_anomaly.csv

ERA5 is for southamerica

In [1]:
import os
import pandas as pd 
import datetime 
import numpy as np

import iris
#from iris.analysis import MEAN
#from iris.analysis.stats import pearsonr
import iris.coord_categorisation

import scipy
from scipy.stats import theilslopes #theil-sen trends

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import f_Iris
#import esmvalcore.preprocessor as ep 

In [2]:
# directories
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

#si estoy trabajando desde el servidor vegeta
ruta_a_archivos_ERA5_mensual = "/pikachu/datos/nadia.testani/Doctorado/datos/ERA5/mon/SAm" #ruta donde estan los archivos con datos de ERA5
ruta_a_archivos_CI = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/paper"
ruta_a_resultados = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2024" #ruta donde se guardan los resultados del analisis de nubosidad

Current working directory: /home/nadia.testani/Doctorado/DoctoradoCodigos/codigos2024


In [3]:
fecha_inicio_climatologia_str = "1983-12-15"
fecha_final_climatologia_str = "2016-11-15"

In [4]:
#Cloud Index
CI = pd.read_csv(f"{ruta_a_archivos_CI}/TCC_monthly_CI_SMN.csv") #from 1961-01-01 until 2021-11-01

#set dates as index
CI = CI.set_index(CI["Unnamed: 0"]) 
CI = CI.drop(["Unnamed: 0"], axis = 1)
CI.index = pd.DatetimeIndex(CI.index)

#estimate regional mean
CI_regional_mean = CI.mean(axis=1)

#filter theil-sen trend
def calculate_theilsen_slope(y):
    slope, intercept, _, _ = theilslopes(y, range(len(y)), alpha=0.95)
    return slope, intercept

CI_detrend = CI_regional_mean.copy()
trend = calculate_theilsen_slope(CI_detrend.values)[0] * np.arange(len(CI_detrend)) + calculate_theilsen_slope(CI_detrend.values)[1]
CI_detrend = CI_detrend - trend

#estimate climatology
CI_detrend_fechas_climatologia = CI_detrend.loc[fecha_inicio_climatologia_str:fecha_final_climatologia_str]
CI_detrend_climatologia = CI_detrend_fechas_climatologia.groupby(CI_detrend_fechas_climatologia.index.month).mean() 

#estimate anomalies
def f_serie_anomalias_mensuales(serie, serie_climatologia):
    serie_anomalia = serie.copy()
    for fecha in serie_anomalia.index:
        for mes in serie_anomalia.index.month.unique():
            if fecha.month == mes:
                serie_anomalia.loc[fecha] = serie.loc[fecha]-serie_climatologia.loc[mes]
    return serie_anomalia

CI_detrend_anom = f_serie_anomalias_mensuales(CI_detrend, CI_detrend_climatologia)

In [5]:
#ERA5
file_dir = ruta_a_archivos_ERA5_mensual
file_name = "new_data.nc"
variable_name = 't2m'
t2m = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 
t2m = t2m[:,0,:,:] #select expver=1

In [6]:
#select dates common to CI 
date_i_CI = CI_detrend_anom.index[0]
date_f_CI = CI_detrend_anom.index[-1]

In [7]:
t2m_CIdates = f_Iris.clip_timerange(t2m, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
#check that it is ok:
time_coord = t2m_CIdates.coord("time") 
time_axis = time_coord.units.num2date(time_coord.core_points())
#print(time_axis)

In [8]:
#detrend
t2m_CIdates_detrend = f_Iris.detrend_theil_sen_cube(t2m_CIdates, dimension='time', method='linear')

/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [14]:
t2m_CIdates_detrend.data #correct function

/home/nadia.testani/miniforge3/envs/base_nadia2/lib/python3.9/site-packages/iris/fileformats/netcdf.py:1052: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if data_dtype.itemsize is 1:
/home/nadia.testani/miniforge3/envs/base_nadia2/lib/python3.9/site-packages/iris/fileformats/netcdf.py:1052: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if data_dtype.itemsize is 1:


RuntimeError: NetCDF: Not a valid ID

In [ ]:
#estimate anomalies

In [12]:
#Select times = times of climatology
#fecha_inicio_climatologia_str = "1983-12-15"
#fecha_final_climatologia_str = "2016-11-15"

time_coord = ERA5_netcdf_1.coord("time") 
time_axis = time_coord.units.num2date(time_coord.core_points())
timei = datetime.datetime(1983, 12, 1, 0, 0) 
timef = datetime.datetime(2016, 11, 1, 0, 0) 

# Create a constraint based on the chosen datetime
time_constraint = iris.Constraint(time=lambda cell: timei <= cell.point <= timef)

# Apply the constraint to slice the cube
ERA5_netcdf_1_datesClimatology = ERA5_netcdf_1.extract(time_constraint)

time_coord = ERA5_netcdf_1_datesClimatology.coord("time")
time_axis = time_coord.units.num2date(time_coord.core_points())

#mean values
# calculate monthly mean
# Categorize the time coordinate by month
iris.coord_categorisation.add_month(ERA5_netcdf_1_datesClimatology, 'time', name='month')

# Calculate the monthly mean for each grid point
monthly_means = ERA5_netcdf_1_datesClimatology.aggregated_by('month', iris.analysis.MEAN)

In [62]:
# Filter monthly means --> anomalies
monthly_means_array = np.repeat(monthly_means.data, len(data_trend_filtered_datesCI.data[:,1,1])/12, axis=0)
ERA5_trend_filtered_anomalies = data_trend_filtered_datesCI.data - monthly_means_array

#I have to delete the last december to calculate the regression

/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [14]:
ERA5_netcdf_detrend

<iris 'Cube' of 2 metre temperature / (K) (time: 768; latitude: 301; longitude: 217)>